In [ ]:
import requests
import pandas as pd
from datetime import datetime, timezone
import base64
import os

# ======== CONFIG ========
BRANCH = "main"
GITHUB_REPO = "SagarFieldElevate/Trial"
UPLOAD_PATH = "uploads"

# Pull secrets from GitHub Actions
GITHUB_TOKEN = os.getenv("GH_TOKEN")
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
BASE_ID = "appFww2fieWK3mKwi"
TABLE_NAME = "Database"

# 1. Fetch trending coins
print("🔍 Fetching trending coins from CoinGecko...")
url = "https://api.coingecko.com/api/v3/search/trending"
resp = requests.get(url)
resp.raise_for_status()
trending_data = resp.json().get("coins", [])

# 2. Convert to DataFrame
coin_list = []
for coin_info in trending_data:
    coin = coin_info.get("item", {})
    data = coin.get("data", {})
    price_change_24h = data.get("price_change_percentage_24h", {})

    coin_list.append({
        "Name": coin.get("name"),
        "Symbol": coin.get("symbol"),
        "ID": coin.get("id"),
        "Market Cap Rank": coin.get("market_cap_rank"),
        "Price (BTC)": coin.get("price_btc"),
        "Score": coin.get("score"),
        "Total Volume": data.get("total_volume"),
        "24h Change (%) USD": price_change_24h.get("usd"),
    })

df = pd.DataFrame(coin_list)

# 3. Save to Excel
timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%d_%H-%M-%S")
filename = f"trending_coins_{timestamp}.xlsx"
df.to_excel(filename, index=False)

# 4. Upload to GitHub
print("📤 Uploading file to GitHub...")
with open(filename, "rb") as f:
    content = base64.b64encode(f.read()).decode()

upload_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
upload_payload = {
    "message": f"Upload trending coins file {timestamp}",
    "content": content,
    "branch": BRANCH
}
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}
upload_resp = requests.put(upload_url, headers=headers, json=upload_payload)
if upload_resp.status_code not in [200, 201]:
    raise Exception(f"❌ GitHub upload failed: {upload_resp.status_code} - {upload_resp.text}")

file_sha = upload_resp.json()["content"]["sha"]
raw_url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{BRANCH}/{UPLOAD_PATH}/{filename}"
print(f"Raw URL: {raw_url}")

# 5. Send to Airtable
print("📬 Sending GitHub file URL to Airtable...")
airtable_url = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json"
}
airtable_payload = {
    "records": [{
        "fields": {
            "Name": timestamp,
            "Trending Excel": [{
                "url": raw_url,
                "filename": filename
            }]
        }
    }]
}
airtable_resp = requests.post(airtable_url, headers=airtable_headers, json=airtable_payload)
print(f"Airtable response: {airtable_resp.status_code} - {airtable_resp.json()}")
if airtable_resp.status_code != 200:
    raise Exception(f"❌ Airtable upload failed: {airtable_resp.status_code} - {airtable_resp.text}")

# 6. Delete from GitHub
print("🧹 Deleting file from GitHub...")
delete_payload = {
    "message": f"Auto-delete {filename} after Airtable upload",
    "sha": file_sha,
    "branch": BRANCH
}
delete_resp = requests.delete(upload_url, headers=headers, json=delete_payload)
if delete_resp.status_code != 200:
    print(f"⚠️ GitHub delete failed: {delete_resp.status_code} - {delete_resp.text}")
else:
    print("✅ GitHub file deleted.")

# 7. Delete local file
os.remove(filename)
print("🧽 Local file deleted.")
print("✅ Workflow complete.")
